# Lab Assignment 2 - Part B: k-Nearest Neighbor Classification
Please refer to the `README.pdf` for full laboratory instructions.


## Problem Statement
In this part, you will implement the k-Nearest Neighbor (k-NN) classifier and evaluate it on two datasets:
- **Lenses Dataset**: A small dataset for contact lens prescription
- **Credit Approval (CA) Dataset**: Credit card application data with binary labels (+/-)

### Your Tasks
1. **Preprocess the data**: Handle missing values and normalize features
2. **Implement k-NN** with L2 distance
3. **Evaluate** on both datasets for different values of k
4. **Discuss** your results

### Datasets
The data files are located in the `credit 2017/` folder:
- `lenses.training`, `lenses.testing`
- `crx.data.training`, `crx.data.testing`
- `crx.names` (describes the features)


## Setup


In [54]:
# Library declarations
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


In [55]:
# Data paths
DATA_PATH = "credit_2017/"

# Load Lenses data
def load_lenses_data():
    """Load the lenses dataset."""
    train_data = np.loadtxt(DATA_PATH + "lenses.training", delimiter=',')
    test_data = np.loadtxt(DATA_PATH + "lenses.testing", delimiter=',')
    
    # First column is ID, last column is label
    X_train = train_data[:, 1:-1]
    y_train = train_data[:, -1]
    X_test = test_data[:, 1:-1]
    y_test = test_data[:, -1]
    
    return X_train, y_train, X_test, y_test

# Load Credit Approval data
def load_credit_data(train_file, test_file):
    """
    Load the Credit Approval dataset.
    Note: This dataset contains missing values (?) and mixed types.
    You will need to preprocess it.
    """
    def _read(path):
        rows=[]
        labels=[]
        with open(path,'r') as f:
            for ln in f:
                ln=ln.strip()
                if not ln:
                    continue
                parts=[p.strip() for p in ln.split(',')]
                rows.append(parts[:-1])
                labels.append(parts[-1])
        return rows, labels
    train_rows, train_labels = _read(train_file)
    test_rows, test_labels = _read(test_file)
    return train_rows, train_labels, test_rows, test_labels
    

# Test loading lenses data
X_train_lenses, y_train_lenses, X_test_lenses, y_test_lenses = load_lenses_data()
print(f"Lenses - Train: {X_train_lenses.shape}, Test: {X_test_lenses.shape}")


Lenses - Train: (18, 3), Test: (6, 3)


## Task 1: Data Preprocessing
For the Credit Approval dataset, you need to:
1. **Handle missing values** (marked with '?'):
   - Categorical features: replace with mode/median
   - Numerical features: replace with label-conditioned mean
2. **Normalize features** using z-scaling:
   $$z_i^{(m)} = \frac{x_i^{(m)} - \mu_i}{\sigma_i}$$

Document exactly how you handle each feature!


In [56]:
import numpy as np

def preprocess_credit_data(train_file, test_file):
    """
    Preprocess the Credit Approval dataset.

    Steps:
    1) Load the data
    2) Handle missing values
       - Categorical: replace '?' with MODE of that column computed on TRAIN only
       - Numerical:   replace '?' with label-conditioned MEAN computed on TRAIN only
                      (mean of that feature among samples with the same label)
    3) Encode categorical variables (fit mappings on TRAIN only)
    4) Normalize numerical features using z-score (TRAIN mean/std), applied to train + test

    Returns:
    --------
    X_train, y_train, X_test, y_test : numpy arrays
    """

    # A1-A15 are features, A16 is label
    # Numerical (continuous): A2, A3, A8, A11, A14, A15 -> 0-based indices: 1,2,7,10,13,14
    num_idx = [1, 2, 7, 10, 13, 14]
    cat_idx = [i for i in range(15) if i not in num_idx]

    # (Index -> Name -> Type)
    feature_docs = [
        (0,  "A1",  "categorical"),
        (1,  "A2",  "numerical"),
        (2,  "A3",  "numerical"),
        (3,  "A4",  "categorical"),
        (4,  "A5",  "categorical"),
        (5,  "A6",  "categorical"),
        (6,  "A7",  "categorical"),
        (7,  "A8",  "numerical"),
        (8,  "A9",  "categorical"),
        (9,  "A10", "categorical"),
        (10, "A11", "numerical"),
        (11, "A12", "categorical"),
        (12, "A13", "categorical"),
        (13, "A14", "numerical"),
        (14, "A15", "numerical"),
    ]

    # 1) Load raw data 
    train_rows, train_labels, test_rows, test_labels = load_credit_data(train_file, test_file)

    # Label encoding: '+' -> 1, '-' -> 0
    y_train = np.array([1 if l.strip() == '+' else 0 for l in train_labels], dtype=int)
    y_test  = np.array([1 if l.strip() == '+' else 0 for l in test_labels], dtype=int)

    # 2) TRAIN only imputation stats
    cat_mode = {}
    for j in cat_idx:
        counts = {}
        for r in train_rows:
            v = r[j]
            if v != '?':
                counts[v] = counts.get(v, 0) + 1
        cat_mode[j] = max(counts, key=counts.get)

    num_mean = {}
    for j in num_idx:
        for c in [0, 1]:
            vals = []
            for r, y in zip(train_rows, y_train):
                if y == c and r[j] != '?':
                    vals.append(float(r[j]))
            if len(vals) == 0:
                vals = [float(r[j]) for r in train_rows if r[j] != '?']
            num_mean[(j, c)] = float(np.mean(vals))

    # 3) Impute missing values
    def impute(rows, y_vec):
        out = []
        for r, c in zip(rows, y_vec):
            r2 = list(r)
            for j in cat_idx:
                if r2[j] == '?':
                    r2[j] = cat_mode[j]
            for j in num_idx:
                if r2[j] == '?':
                    r2[j] = str(num_mean[(j, int(c))])
            out.append(r2)
        return out

    train_imp = impute(train_rows, y_train)
    test_imp  = impute(test_rows, y_test)

    # 4) Encode categoricals 
    cat_maps = {}
    for j in cat_idx:
        uniq = sorted({r[j] for r in train_imp})
        cat_maps[j] = {val: idx for idx, val in enumerate(uniq)}

    def encode(rows):
        X = np.zeros((len(rows), 15), dtype=float)
        for i, r in enumerate(rows):
            for j in range(15):
                if j in num_idx:
                    X[i, j] = float(r[j])
                else:
                    X[i, j] = float(cat_maps[j][r[j]])
        return X

    X_train = encode(train_imp)
    X_test  = encode(test_imp)

    # 5) Z-normalize numeric columns
    X_train, X_test = z_normalize(X_train, X_test, num_idx)

    return X_train, y_train, X_test, y_test


def z_normalize(X_train, X_test, feature_indices):
    X_train = X_train.astype(float).copy()
    X_test  = X_test.astype(float).copy()
    for j in feature_indices:
        mu = float(np.mean(X_train[:, j]))
        sigma = float(np.std(X_train[:, j]))
        if sigma == 0.0:
            sigma = 1.0
        X_train[:, j] = (X_train[:, j] - mu) / sigma
        X_test[:, j]  = (X_test[:, j] - mu) / sigma
    return X_train, X_test


## Task 2: Implement k-NN Classifier
Implement k-NN with L2 (Euclidean) distance:
$$\mathcal{D}_{L2}(\mathbf{a}, \mathbf{b}) = \sqrt{\sum_i (a_i - b_i)^2}$$

For **categorical attributes**, use:
- Distance = 1 if values are different
- Distance = 0 if values are the same


In [57]:
def l2_distance(a, b, num_idx=None, cat_idx=None):
    """
    Compute L2 (Euclidean) distance between two vectors.
    
    Parameters:
    -----------
    a, b : numpy arrays of same shape
    
    Returns:
    --------
    distance : float
    """
    a = np.asarray(a)
    b = np.asarray(b)
    total = 0.0

    if num_idx is None:
        # default to Euclidean over all dims
        diff = a - b
        return float(np.sqrt(np.sum(diff * diff)))

    for j in num_idx:
        d = float(a[j] - b[j])
        total += d * d
    for j in cat_idx:
        total += 0.0 if int(a[j]) == int(b[j]) else 1.0
    return float(np.sqrt(total))


def knn_predict(X_train, y_train, X_test, k, num_idx=None, cat_idx=None):
    """
    Predict labels for test data using k-NN.
    
    Parameters:
    -----------
    X_train : numpy array of shape (n_train, n_features)
    y_train : numpy array of shape (n_train,)
    X_test : numpy array of shape (n_test, n_features)
    k : int, number of neighbors
    
    Returns:
    --------
    predictions : numpy array of shape (n_test,)
    """
    X_train = np.asarray(X_train)
    y_train = np.asarray(y_train).astype(int)
    X_test = np.asarray(X_test)

    preds = np.zeros((X_test.shape[0],), dtype=int)

    for i, x in enumerate(X_test):
        # compute distances
        dists = np.zeros((X_train.shape[0],), dtype=float)
        for j, xt in enumerate(X_train):
            dists[j] = l2_distance(x, xt, num_idx=num_idx, cat_idx=cat_idx)

        nn_idx = np.argsort(dists)[:k]
        nn_labels = y_train[nn_idx]

        # nearest neighbor vote
        preds[i] = np.bincount(nn_labels).argmax()        
        nn_idx = np.argsort(dists)[:k]
        nn_labels = y_train[nn_idx]
        preds[i] = np.bincount(nn_labels).argmax()

    return preds




def compute_accuracy(y_true, y_pred):
    """
    Compute classification accuracy.
    
    Returns:
    --------
    accuracy : float (between 0 and 1)
    """
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean(y_true == y_pred))

## Task 3: Evaluate on Lenses Dataset
Test your k-NN implementation on the Lenses dataset for different values of k.


In [58]:
# TODO: Evaluate k-NN on Lenses dataset
# Try different values of k (e.g., 1, 3, 5, 7)

k_values = [1, 3, 5, 7]
lenses_results = []

for k in k_values:
    predictions = knn_predict(X_train_lenses, y_train_lenses, X_test_lenses, k)
    accuracy = compute_accuracy(y_test_lenses, predictions)
    lenses_results.append((k, accuracy))
    print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 1.0000
k=3: Accuracy = 1.0000
k=5: Accuracy = 0.5000
k=7: Accuracy = 0.8333


## Task 4: Evaluate on Credit Approval Dataset
First preprocess the data, then evaluate k-NN.


In [59]:
# TODO: Preprocess Credit Approval data
X_train_credit, y_train_credit, X_test_credit, y_test_credit = preprocess_credit_data(
    DATA_PATH + "crx.data.training",
    DATA_PATH + "crx.data.testing"
)

print(f"Credit Approval - Train: {X_train_credit.shape}, Test: {X_test_credit.shape}")


Credit Approval - Train: (552, 15), Test: (138, 15)


In [60]:
# TODO: Evaluate k-NN on Credit Approval dataset
k_values = [1, 3, 5, 7]
credit_results = []

for k in k_values:
    predictions = knn_predict(X_train_credit, y_train_credit, X_test_credit, k)
    accuracy = compute_accuracy(y_test_credit, predictions)
    credit_results.append((k, accuracy))
    print(f"k={k}: Accuracy = {accuracy:.4f}")


k=1: Accuracy = 0.7464
k=3: Accuracy = 0.7826
k=5: Accuracy = 0.8043
k=7: Accuracy = 0.7971


## Summary and Discussion

### Results Table

| Dataset | k=1 | k=3 | k=5 | k=7 |
|---------|-----|-----|-----|-----|
| Lenses | 100.00 | 100.00 | 50.00 | 83.33 |
| Credit Approval | 74.64 | 78.26 | 80.43 | 79.71 |

### Discussion
*Answer these questions:*
1. Which value of k works best for each dataset? Why do you think that is?
    -  For the Lenses dataset a lower k value works best. This is because the dataset is small and limited meaning edge cases will be misclasified. 
    - For the credit score, higher k values yield better results. This is due to the dataset being much larger, and having more features. This allows the data to seperate from classes and allow machine learning methods to understand the underlying patterns between features.
2. How did preprocessing affect your results on the Credit Approval dataset?
    - Preproccessing is essential in eliminating bias and noise. Most importantly scaling the data allows for an "even playing field" between data points. Larger values are not overshadowing the smaller more subtle patterns. 
3. What are the trade-offs of using different values of k?
    - When you have a limited dataset and a high number of K nieghbors, the edge cases tend to get misclassified. It applies to larger datasets as well, however, the effects are mitigated because there are more datapoints to reflect the real world. The best course of action would be to preproccess the dataset, then begin training the model. A method such as Bayesian Optimization can be employed to automatically tune a model to the best k nearest neighbors.
4. What did you learn from this exercise?
    -   This exercise was very interesting. Preprocessing is the most essential stage in machine leanring. If not implemented correctly the ML model can crash. It was very cool implementing the statistical methods from scratch such as knn with l2 distance




